### Phenotype Table for Initial Set of Training Data

In [1]:
import datetime
import pandas as pd
import numpy as np

#### Import dataset downloaded from betydb in R

In [2]:
df_0 = pd.read_csv('../data/raw/mac_season_4.csv')
df_0.head()

/opt/miniconda3/envs/terraref-datasets/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,26,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
0,1,0,traits,6001958927,6.000000e+09,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI181083,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
1,2,0,traits,6001958928,6.000000e+09,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI564163,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
2,3,0,traits,6001958931,6.000000e+09,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI52606,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
3,4,0,traits,6001958933,6.000000e+09,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI533792,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
4,5,0,traits,6001958936,6.000000e+09,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI535794,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


In [3]:
df_0.columns

Index(['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id',
       'treatment_id', 'sitename', 'city', 'lat', 'lon', 'scientificname',
       'commonname', 'genus', 'species_id', 'cultivar_id', 'author',
       'citation_year', 'treatment', 'date', 'time', 'raw_date', 'month',
       'year', 'dateloc', 'trait', 'trait_description', 'mean', 'units', 'n',
       'statname', 'stat', 'notes', 'access_level', 'cultivar', 'entity',
       'method_name', 'view_url', 'edit_url'],
      dtype='object')

In [4]:
df_1 = df_0.drop(labels=['Unnamed: 0', 'checked', 'citation_id', 'city', 'scientificname', 'commonname', 'genus',
                        'species_id', 'author', 'citation_year', 'trait_description', 'units', 'n', 'statname',
                        'stat', 'access_level', 'view_url', 'edit_url'], axis=1)

#### Extract Range and Column Values

In [5]:
df_2 = df_1.copy()

In [6]:
df_2['range'] = df_2['sitename'].str.extract("Range (\d+)").astype(int)
df_2['column'] = df_2['sitename'].str.extract("Column (\d+)").astype(int)

#### Convert table to wide format
* Each trait should have its own column
* Rename `mean` column to `value` for easier understanding

In [7]:
df_3 = df_2.rename({'mean': 'value'}, axis=1)

In [8]:
traits_to_keep = ['leaf_temperature', 'ambient_humidity', 'proximal_air_temperature', 'surface_temperature',
                  'aboveground_dry_biomass', 'canopy_height', 'flag_leaf_emergence_time', 'flowering_time',
                  'canopy_cover']

In [9]:
empty_df = pd.DataFrame(data=df_3, index=df_3.index, columns=traits_to_keep)

In [10]:
df_4 = pd.concat([df_3, empty_df.reindex(df_3.index)], axis=1)

#### Drop more unecessary (at this time) columns

In [11]:
df_5 = df_4.drop(labels=['result_type', 'treatment_id', 'treatment', 'dateloc'], axis=1)

#### Populate empty columns with available values

In [12]:
# This is very slow - needs refactoring for .py script and reproducible notebook 

# run_slow_stuff = False

# if run_slow_stuff:

counter = 0

for index, row in df_5.iterrows():
    if counter % 1000 == 0:
        print(counter)            
        counter += 1
    for trait in traits_to_keep:
        if row['trait'] == trait:                
            df_5.loc[index, [trait]] = row['value']

0
0
id                                                            6001958927
site_id                                                       6000005673
sitename                    MAC Field Scanner Season 4 Range 11 Column 5
lat                                                              33.0749
lon                                                             -111.975
cultivar_id                                                   6000000730
date                                       2017 Jun 14 (America/Phoenix)
time                                       [time unspecified or unknown]
raw_date                                       2017-06-15 00:00:00 -0500
month                                                                  6
year                                                                2017
trait                                           leaf_desiccation_present
value                                                                  0
notes                                          

4000
4000
id                                                                 6002018919
site_id                                                            6000005947
sitename                         MAC Field Scanner Season 4 Range 45 Column 4
lat                                                                   33.0761
lon                                                                  -111.975
cultivar_id                                                        6000000962
date                                                              2017 Aug 21
time                                               13:29:00 (America/Phoenix)
raw_date                                            2017-08-21 15:29:00 -0500
month                                                                       8
year                                                                     2017
trait                                                        leaf_temperature
value                                                 

8000
8000
id                                                                 6002020358
site_id                                                            6000005944
sitename                         MAC Field Scanner Season 4 Range 45 Column 3
lat                                                                   33.0761
lon                                                                  -111.975
cultivar_id                                                        6000000964
date                                                              2017 Aug 23
time                                               10:30:00 (America/Phoenix)
raw_date                                            2017-08-23 12:30:00 -0500
month                                                                       8
year                                                                     2017
trait                                                                     vH+
value                                                 

12000
12000
id                                                            6002245821
site_id                                                       6000005465
sitename                    MAC Field Scanner Season 4 Range 6 Column 15
lat                                                              33.0747
lon                                                             -111.975
cultivar_id                                                   6000000473
date                                                         2017 Apr 28
time                                          12:00:00 (America/Phoenix)
raw_date                                       2017-04-28 14:00:00 -0500
month                                                                  4
year                                                                2017
trait                                                surface_temperature
value                                                            22.4661
notes                                  

17000
17000
id                                                             6004530700
site_id                                                        6000005703
sitename                    MAC Field Scanner Season 4 Range 11 Column 15
lat                                                               33.0749
lon                                                              -111.975
cultivar_id                                                    6000000871
date                                                          2017 Jun 20
time                                           12:00:00 (America/Phoenix)
raw_date                                        2017-06-20 14:00:00 -0500
month                                                                   6
year                                                                 2017
trait                                                      leaf_angle_chi
value                                                             1.96749
notes                     

...

353000
353000
id                                                            6004793021
site_id                                                       6000005331
sitename                    MAC Field Scanner Season 4 Range 50 Column 7
lat                                                              33.0763
lon                                                             -111.975
cultivar_id                                                   6000000703
date                                                          2017 Aug 9
time                                          12:00:00 (America/Phoenix)
raw_date                                       2017-08-09 14:00:00 -0500
month                                                                  8
year                                                                2017
trait                                                      canopy_height
value                                                                323
notes                                

358000
358000
id                                                            6004586583
site_id                                                       6000005914
sitename                    MAC Field Scanner Season 4 Range 44 Column 9
lat                                                              33.0761
lon                                                             -111.975
cultivar_id                                                   6000000555
date                                                         2017 Jul 20
time                                          12:00:00 (America/Phoenix)
raw_date                                       2017-07-20 14:00:00 -0500
month                                                                  7
year                                                                2017
trait                                                    leaf_angle_beta
value                                                            1.57115
notes                                

362000
362000
id                                                             6004519120
site_id                                                        6000005256
sitename                    MAC Field Scanner Season 4 Range 13 Column 13
lat                                                                33.075
lon                                                              -111.975
cultivar_id                                                    6000000896
date                                                          2017 Jun 15
time                                           12:00:00 (America/Phoenix)
raw_date                                        2017-06-15 14:00:00 -0500
month                                                                   6
year                                                                 2017
trait                                                      leaf_angle_chi
value                                                             2.03471
notes                   

366000
366000
id                                                             6004493690
site_id                                                        6000005836
sitename                    MAC Field Scanner Season 4 Range 36 Column 10
lat                                                               33.0758
lon                                                              -111.975
cultivar_id                                                    6000000975
date                                                          2017 May 25
time                                           12:00:00 (America/Phoenix)
raw_date                                        2017-05-25 14:00:00 -0500
month                                                                   5
year                                                                 2017
trait                                                    leaf_angle_alpha
value                                                             2.34422
notes                   

370000
370000
id                                                            6004499165
site_id                                                       6000005595
sitename                    MAC Field Scanner Season 4 Range 22 Column 5
lat                                                              33.0753
lon                                                             -111.975
cultivar_id                                                   6000000889
date                                                          2017 Jun 4
time                                          12:00:00 (America/Phoenix)
raw_date                                       2017-06-04 14:00:00 -0500
month                                                                  6
year                                                                2017
trait                                                    leaf_angle_mean
value                                                            0.41698
notes                                

In [13]:
df_6 = df_5.set_index('sitename')

#### Drop some columns that are redundant or can be explained in data dictionary
* `month`
* `year`
* `notes`
* `trait` - now have trait values in wide format (one column per trait requested for this iteration of dataset)
* `method_name`
* `notes` 

In [14]:
df_6.drop(labels=['month', 'year', 'notes', 'trait', 'method_name', 'notes'], axis=1, inplace=True)

#### Read in `df_6` 

In [15]:
# df_6 = pd.read_csv(('../data/processed/pheno-table_populated_traits_2019-11-18T071126.csv')

#### Add growing degree days

#### Add max canopy height

#### Re-order column names

#### Change plots to index

#### Add planting date column

In [16]:
# Update df_* 
# df_6 will be renamed

timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
output_filename = f'pheno-table_{timestamp}.csv'.replace(':', '')
df_6.to_csv(f'../data/processed/{output_filename}')